# LAPD officers

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from random import randint
from time import sleep
from tqdm import tqdm
import time

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None

In [4]:
today = pd.Timestamp("today").strftime("%Y-%m-%d")

## Read data

#### U.S. Census Bureau surnames

In [5]:
# https://www.census.gov/data/developers/data-sets/surnames.html
# Any surname listed more than 100 times on the 2000 or 2010 census

In [6]:
names_df = pd.read_csv("../surnames/data/raw/raw_census_surnames_2000_2010.csv")

In [7]:
pop_names_df = names_df.query("rank < 5000")
# pop_names_df = names_df

In [8]:
names = list(pop_names_df["name"].unique())

In [9]:
len(names)

5202

#### Loop and snag names

In [10]:
headers = {
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.9,es;q=0.8",
    "Connection": "keep-alive",
    "If-None-Match": 'W/"3322-3a7IO0W3pE9Z8JxYxN1y7ZBDfCM-gzip"',
    "Referer": "https://watchthewatchers.net/",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36",
    "sec-ch-ua": '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
}

In [11]:
%%time

responses = []

for name in tqdm(names):
    response = requests.get(f"https://watchthewatchers.net/api/search/{name}", headers=headers)
    responses.append(pd.DataFrame(response.json()))

100%|█████████████████████████████████████| 5202/5202 [1:09:23<00:00,  1.25it/s]

CPU times: user 3min 4s, sys: 21.9 s, total: 3min 26s
Wall time: 1h 9min 23s


In [12]:
df = pd.concat(responses).drop_duplicates(subset="id")

#### Dates

In [13]:
df["created"] = pd.to_datetime(df["createdAt"]).dt.strftime("%Y-%m-%d")
df["updated"] = pd.to_datetime(df["updatedAt"]).dt.strftime("%Y-%m-%d")
df["hired"] = pd.to_datetime(df["yearOfHire"]).dt.strftime("%Y-%m-%d")
df["name"] = df["name"].str.title()
df["image_url"] = "https://watchthewatchers.net/headshots/" + df["imagePath"]

In [14]:
df = df.drop(["createdAt", "updatedAt", "imagePath", "yearOfHire"], axis=1)

In [15]:
df.ethnicity.value_counts(normalize=True).round(2) * 100

Hispanic           53.0
White              26.0
Black               9.0
Asian               8.0
Filipino            2.0
Other               0.0
American Indian     0.0
Name: ethnicity, dtype: float64

In [18]:
len(df)

In [21]:
df.query('name.str.contains("Matthew")')

,id,name,serialNumber,ethnicity,gender,area,rank,created,updated,hired,image_url
16,1f93efca-307b-4881-aa97-93b04e61a4a6,Matthew Johnson,37043,White,Male,Foothill Division,Detective I,2023-03-18,2023-03-18,2003-01-01,https://watchthewatchers.net/headshots/JOHNSONMATTHEW-37043.jpg
17,05d842b0-a711-46c1-afc0-477aef1b2a82,Matthew Williams,36359,White,Male,Metro Division,Police Officer III,2023-03-18,2023-03-18,2002-01-01,https://watchthewatchers.net/headshots/WILLIAMSMATTHEWO-36359.jpg
71,596366b9-13dd-4ff1-85e8-e9d78bcdc95a,Matthew Martinez,32880,Hispanic,Male,Community Safety Partnership Bureau,Police Officer III,2023-03-18,2023-03-18,1996-01-01,https://watchthewatchers.net/headshots/MARTINEZMATTHEWA-32880.jpg
9,4497b885-077a-4b8d-9be3-ca13b008a6e9,Matthew Anderson,40073,White,Male,Van Nuys Division,Police Officer III,2023-03-18,2023-03-18,2009-01-01,https://watchthewatchers.net/headshots/ANDERSONMATTHEW-40073.jpg
27,32b03cd0-4fdf-496b-a25b-04b2651f7c6a,Matthew Martin,43648,White,Male,Southeast Division,Police Officer II,2023-03-18,2023-03-18,2018-01-01,https://watchthewatchers.net/headshots/MARTINMATTHEWJ-43648.jpg
37,98baf1ce-6b79-496b-a203-5acb499d89b5,Matthew Lee,33041,Asian,Male,ADSD,Police Officer III,2023-03-18,2023-03-18,1996-01-01,https://watchthewatchers.net/headshots/LEEMATTHEWD-33041.jpg
41,bcc8b634-a857-4ff4-a204-134759a67599,Matthew Perez,45162,Black,Male,West Los Angeles Division,Police Officer I,2023-03-18,2023-03-18,2022-01-01,https://watchthewatchers.net/headshots/PEREZMATTHEWJ-45162.jpg
13,efa74751-e38b-4286-9f4f-dcc2bd3159d0,Matthew Whitelaw,38480,White,Male,Mission Division,Police Officer II,2023-03-18,2023-03-18,2007-01-01,https://watchthewatchers.net/headshots/WHITELAWMATTHEWC-38480.jpg
11,eb7ee755-bf21-4456-a225-eb789a06c21f,Matthew Clark,41636,White,Male,South Bureau Homicide Division,Police Officer II,2023-03-18,2023-03-18,2014-01-01,https://watchthewatchers.net/headshots/CLARKMATTHEW-41636.jpg
5,b56569d3-8cb2-4ef4-a8dd-f1345485f116,Matthew Mitchell,37402,White,Male,Training Division,Police Officer III,2023-03-18,2023-03-18,2005-01-01,https://watchthewatchers.net/headshots/MITCHELLMATTHEWR-37402.jpg


---

## Export

In [17]:
df.to_csv("data/processed/lapd-officers-roster.csv", index=False)